In [83]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import sys

###Familirization

1. The data we got is relatively sparse. This means that there are a lot of people who liked only one other person. This is good for the nmf (WHY????????)

 while it is bad for the min-hashing as the point of min-hashing is to shrink the potential size of the data from (amount of users * amount of liked items) to (amount of users * amount of hashes).

We also noticed that most users dislike people more than that they like people, as in "false" is significantly more common in the "is_like" column. below this cell is a graph of the distribution of likes and dislikes. 


2. In terms of the different types of people. some people predominently like people that are already liked by a lot of other people, so being popular makes you more popular. 

In [84]:
train_file_path = 'lab2_train.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')

test_file_path = 'lab2_test.csv'

test_data = pd.read_csv(test_file_path, delimiter=',')
print(train_data)
print(train_data.shape[0])


       user_from_id  user_to_id  is_like is_match
0              1136        3141    False    False
1              2424        3174    False    False
2              1300        3590    False    False
3               800        2736    False    False
4               883         437    False    False
...             ...         ...      ...      ...
76387          2376        3057    False    False
76388          1163         933    False    False
76389          2770        3324    False    False
76390           879         785    False    False
76391           291         470    False    False

[76392 rows x 4 columns]
76392


The hashfunction class from the weblab assignment:

In [85]:
class HashFunction:
    """
    This HashFunction class can be used to create an unique hash given an alpha and beta.
    """
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def hashf(self, x, n):
        """
        Returns a hash given integers x and n.
        :param x: The value to be hashed
        :param n: The number of unique ids of all sets
        :return: The hashed value x given alpha and beta
        """
        
        hash_value = 0
        hash_value =  (self.alpha * x + self.beta) % n
        return hash_value

The compute signature function from the weblab assignment:

In [86]:
def compute_signature(hashes: list[HashFunction], ids: list[set[int]]):
    """
    This function will calculate the MinHash signature matrix from our sets of ids
    using the list of hash functions (hashes)
    :param hashes: The list of hash functions of arbitrary length
    :param ids: The list of sets of ids
    :return: The MinHash signature matrix for the given sets of ids
    """
    
    result = np.full((len(hashes), len(ids)), sys.maxsize)
    space = set().union(*ids)
    sorted_space = sorted(space)
    for p, ID in enumerate(sorted_space):
        arr = []
        for k, has in enumerate(hashes):
            arr.append(hashes[k].hashf(p, len(sorted_space)))
        for j in range(len(ids)):
            if (ID in ids[j]):
                for k, has in enumerate(hashes):
                    result[k,j] = min(arr[k], result[k,j])
    return result





    
    # START ANSWER
    # END ANSWER
    
    return result


### preprocessing
considering that our data consist of a bunch of binary values representing if person A liked person B or not, it didn't make a lot of sense to do traditional normilization. We did however consider if we should remove the outliers in the sense that we
(NOTE TO DELIA: expand on this based on what we did to the outliers). Furthermore according to mattermost the is_match column should not be considered, so we removed it from the data. At least for the min-hashing we only considered the "True" values in the "is_like" column, as we wanted to build a recommender system based in which users other users liked.

In [87]:
#Remove the is_match column since that is not considered
train_data = train_data.drop('is_match', axis=1)

#sort the data by user_from_id
sorted_train_data = train_data.sort_values('user_from_id')

#filter the data by is_like = True, since we want to know for each user which users they have liked.
#The point point of a reccomender system is to recommend things and not to dissuade people from things.
filtered_sorted_train_data = sorted_train_data[sorted_train_data['is_like'] == True]

#group the data by user_from_id to create the sets of liked users for each user. 
#the aggregation created an outer list around the sets for some reason so I had to flatten it.
grouped_train_data = filtered_sorted_train_data.groupby('user_from_id')
set_grouped_train_data = grouped_train_data.agg({'user_to_id': lambda x: set(x)})
list_set_grouped_train_data = set_grouped_train_data.values.flatten().tolist()

#a print to show what the data looks like.
print(set_grouped_train_data)



               user_to_id
user_from_id             
1             {837, 3046}
2                  {3058}
3                  {1179}
4             {448, 3058}
5                    {15}
...                   ...
3682               {3185}
3688               {3238}
3689               {3366}
3707               {3444}
3714               {3568}

[2107 rows x 1 columns]


Prints the following things in order:
- the id of the first user
- the id of the second user
- the id's liked by the first user
- the id's liked by the second user
- the hash signature of the first user
- the hash signature of the second user

In [88]:
def print_stuff(res, hash_amount, similarity_ratio, neighbours_amount):
    for i in range(res.shape[1]):  # generate pairs
        neighbours = []
        for j in range(i + 1,res.shape[1]): 
            same = 0
            
            i_array = []
            j_array = []
            for k in range(hash_amount):
                    
                i_array.append(res[k][i])
                j_array.append(res[k][j])
                if (res[k][i] == res[k][j]):
                    same += 1
            if (same/hash_amount >= similarity_ratio and same/hash_amount < 1):
                neighbours.append((res[k][j], same/hash_amount, j)) 
            else:
                pass
        #makes sure to only keep the neighbours_amount closest neighbours
        while (len(neighbours) > neighbours_amount):
                lowest_similarity_ratio = 2
                lowest_similarity_ratio_index = -1
                for k in range(len(neighbours)):
                    if (neighbours[k][1] < lowest_similarity_ratio):
                        lowest_similarity_ratio = neighbours[k][1]
                        lowest_similarity_ratio_index = k
                neighbours.pop(lowest_similarity_ratio_index)
        for k in range(len(neighbours)):
            neighbour_index = neighbours[k][2] 
            #we add a plus one to all the indices since the indices start at 0 but the user ids start at 1
            print(i+1)
            print(neighbour_index + 1)
            print(set_grouped_train_data['user_to_id'].values[i])
            print(set_grouped_train_data['user_to_id'].values[neighbour_index])
            print(i_array)
            print(neighbours[k][0])
            print(f"recommended users to user {i+1}:" + str(set_grouped_train_data['user_to_id'].values[neighbour_index] - set_grouped_train_data['user_to_id'].values[i]))
            print(" ")
              

The performance of the recommender depends on the amount of hash functions used. the signature matrix returned by the compute_signature_matrix function is a matrix of size (n_hash_functions, m_users). so each column represents the hash of the set of "liked users" of a user. the chance that a specific minimum hash value is the same in different sets is equal to the jaccard similarity. This means that, assuming that we use enough hash functions, the amount of identical minimum values in two columns of the signature matrix is equal to the jaccard similarity of the two sets. The reason why we have to expirement with the amount of hash functions, is because using to few could cause two sets to appear more similar than they really are due to chance. This is demonstrated in the following examples as the fewer hash functions we use, the more non similar sets show up

In terms of how many nearest neighbours to use, it's really up to preference. adding more nearest neighbours will give you more recommendations, but eventually you start dealing with sets that are less and less similar. For this reason we felt that adding a similarity threshold was a good idea in combination with a nearest neighbour amount of 5. This way we can make sure that we only recommend sets of liked people that are at least somewhat similar to the original.

In [89]:
hashes = []
#amount of hash functions
hash_amount = 10

#threshold for how similar two sets have to be to be considered similar. 
#This way we don't recommend incorrect users because one of the sets is an outlier and has no close nearest neighbours 
similarity_ratio = 0.4

#amount of neighbours to recommend
neighbours_amount = 3

#generate the hash functions
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

#compute the signature matrix
res = compute_signature(hashes, list_set_grouped_train_data)

#prints stuff
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
1937
{837, 3046}
{837}
[332, 65, 276, 872, 118, 298, 84, 374, 838, 915]
1467
recommended users to user 1:set()
 
1
1982
{837, 3046}
{837}
[332, 65, 276, 872, 118, 298, 84, 374, 838, 915]
1467
recommended users to user 1:set()
 
1
1983
{837, 3046}
{1869, 837}
[332, 65, 276, 872, 118, 298, 84, 374, 838, 915]
1467
recommended users to user 1:{1869}
 
2
4
{3058}
{448, 3058}
[1463, 713, 226, 74, 1624, 514, 1539, 1682, 1330, 594]
594
recommended users to user 2:{448}
 
2
60
{3058}
{1840}
[1463, 713, 226, 74, 1624, 514, 1539, 1682, 1330, 594]
594
recommended users to user 2:{1840}
 
2
520
{3058}
{1840}
[1463, 713, 226, 74, 1624, 514, 1539, 1682, 1330, 594]
594
recommended users to user 2:{1840}
 
3
938
{1179}
{3595}
[644, 944, 1204, 1133, 280, 22, 975, 1358, 1537, 591]
591
recommended users to user 3:{3595}
 
3
2029
{1179}
{1483, 1820, 1179}
[644, 944, 1204, 1133, 280, 22, 975, 1358, 1537, 591]
591
recommended users to user 3:{1483, 1820}
 
4
932
{448, 3058}
{3058}
[1463, 643, 226, 74, 1514

KeyboardInterrupt: 

In [ ]:
hashes = []
#amount of hash functions
hash_amount = 20
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
936
{837, 3046}
{3046}
[455, 992, 662, 506, 1163, 160, 460, 55, 118, 373, 88, 229, 561, 612, 923, 541, 153, 233, 140, 170]
170
 
1
1937
{837, 3046}
{837}
[455, 992, 662, 506, 1163, 160, 460, 55, 118, 373, 88, 229, 561, 612, 923, 541, 153, 233, 140, 170]
1536
 
1
1982
{837, 3046}
{837}
[455, 992, 662, 506, 1163, 160, 460, 55, 118, 373, 88, 229, 561, 612, 923, 541, 153, 233, 140, 170]
1536
 
1
1983
{837, 3046}
{1869, 837}
[455, 992, 662, 506, 1163, 160, 460, 55, 118, 373, 88, 229, 561, 612, 923, 541, 153, 233, 140, 170]
1170
 
17
64
{1424, 2, 2779, 92}
{2, 92, 391}
[263, 1, 17, 228, 697, 153, 244, 527, 109, 31, 238, 38, 402, 627, 725, 112, 485, 23, 5, 220]
220
 
21
85
{506, 3262}
{506, 2907}
[741, 69, 1605, 20, 777, 1404, 184, 515, 391, 931, 256, 58, 460, 318, 155, 1020, 672, 103, 821, 795]
795
 
21
233
{506, 3262}
{3262}
[741, 69, 1605, 20, 777, 1404, 184, 515, 391, 931, 256, 58, 460, 318, 155, 1020, 672, 103, 821, 795]
1435
 
22
566
{709, 393, 2186, 91, 2940, 2269}
{393, 2186, 2699, 

In [ ]:
hashes = []
hash_amount = 50
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)

1
936
{837, 3046}
{3046}
[423, 857, 10, 730, 747, 799, 1549, 83, 849, 738, 838, 999, 754, 500, 295, 1113, 200, 390, 746, 389, 62, 330, 993, 935, 32, 805, 337, 368, 398, 1076, 503, 185, 28, 137, 434, 728, 926, 425, 1176, 88, 1019, 768, 442, 1515, 897, 351, 273, 990, 217, 265]
803
 
1
1937
{837, 3046}
{837}
[423, 857, 10, 730, 747, 799, 1549, 83, 849, 738, 838, 999, 754, 500, 295, 1113, 200, 390, 746, 389, 62, 330, 993, 935, 32, 805, 337, 368, 398, 1076, 503, 185, 28, 137, 434, 728, 926, 425, 1176, 88, 1019, 768, 442, 1515, 897, 351, 273, 990, 217, 265]
265
 
1
1982
{837, 3046}
{837}
[423, 857, 10, 730, 747, 799, 1549, 83, 849, 738, 838, 999, 754, 500, 295, 1113, 200, 390, 746, 389, 62, 330, 993, 935, 32, 805, 337, 368, 398, 1076, 503, 185, 28, 137, 434, 728, 926, 425, 1176, 88, 1019, 768, 442, 1515, 897, 351, 273, 990, 217, 265]
265
 
2
4
{3058}
{448, 3058}
[1050, 247, 1453, 1606, 430, 1399, 1060, 1134, 839, 1107, 458, 1407, 574, 102, 752, 118, 347, 1698, 407, 1256, 1481, 215, 279, 1701

In [ ]:
hashes = []
hash_amount = 100
similarity_ratio = 0.4
neighbours_amount = 5
for i in range(hash_amount):
    hashes.append(HashFunction(random.randint(1,10000), random.randint(1,10000)))

res = compute_signature(hashes, list_set_grouped_train_data)
print_stuff(res, hash_amount, similarity_ratio, neighbours_amount)


1
936
{837, 3046}
{3046}
[873, 24, 486, 899, 567, 1377, 433, 325, 104, 476, 107, 712, 772, 653, 74, 1450, 206, 496, 32, 1212, 342, 202, 127, 724, 786, 365, 193, 545, 825, 1337, 680, 248, 811, 94, 471, 806, 108, 338, 441, 827, 363, 545, 840, 512, 1118, 319, 488, 622, 230, 380, 196, 117, 127, 118, 198, 1047, 393, 257, 978, 821, 279, 1072, 291, 423, 612, 649, 498, 347, 1483, 737, 1167, 489, 325, 1181, 94, 297, 725, 56, 264, 131, 321, 633, 129, 944, 456, 460, 896, 909, 245, 1360, 344, 1079, 946, 35, 54, 435, 652, 1409, 597, 364]
1416
 
1
1937
{837, 3046}
{837}
[873, 24, 486, 899, 567, 1377, 433, 325, 104, 476, 107, 712, 772, 653, 74, 1450, 206, 496, 32, 1212, 342, 202, 127, 724, 786, 365, 193, 545, 825, 1337, 680, 248, 811, 94, 471, 806, 108, 338, 441, 827, 363, 545, 840, 512, 1118, 319, 488, 622, 230, 380, 196, 117, 127, 118, 198, 1047, 393, 257, 978, 821, 279, 1072, 291, 423, 612, 649, 498, 347, 1483, 737, 1167, 489, 325, 1181, 94, 297, 725, 56, 264, 131, 321, 633, 129, 944, 456, 460, 89